I aim to calculate the error ellipse from an given 2D error matrix.
I will use the radio source 1901+319 as an example to illustrate the calculation algrithm and also check my result with Sebastien's.
For 1901+319, the major (M) and minor (m) axes, and positional angle (PA) in the opa15 solution are
M = 0.521 mas
m = 0.229 mas
PA = 125 degree

In [1]:
ra_err = 0.3356
dec_err = 0.4337
ra_dec_corr = -0.792

The first method is to use the linear algebra tools.

In [2]:
import numpy as np

cov = ra_dec_corr * ra_err * dec_err
cov_mat = np.array([[ra_err**2, cov], [cov, dec_err**2]])

In [8]:
from numpy.linalg import eig

eig_val, eig_vec = eig(cov_mat)
eig_vec

(array([0.02906734, 0.27165571]), array([[-0.80965926,  0.58690023],
        [-0.58690023, -0.80965926]]))

In [17]:
M2 = np.max(eig_val)
m2 = np.min(eig_val)

M, m = np.sqrt(M2), np.sqrt(m2)

index = np.where(eig_val == M2)[0][0]

vec_M = eig_vec[:,index]

nor_vec = np.array([0, 1])
# nor_vec = np.array([1, 0])
pa0 = np.rad2deg(np.arccos(np.dot(vec_M, nor_vec)))

print(pa0)

if vec_M[0] > 0:
    pa = 180 - pa0
else:
    pa = pa0

print("M = %.3f mas\n"
      "m = %.3f mas\n"
      "PA = %.0f degree" % (M, m, pa))

vec_M, pa0

144.06265350727185
M = 0.521 mas
m = 0.170 mas
PA = 144 degree


(array([-0.58690023, -0.80965926]), 144.06265350727185)

Also, we can use some formulas, as in Mignard et. al 2018.

In [21]:
from numpy import sqrt

M = sqrt(0.5 * (ra_err**2 + dec_err**2 +
                              sqrt((ra_err**2 - dec_err**2)**2 +
                                   (2*ra_err*dec_err*ra_dec_corr)**2)))

m = sqrt(0.5 * (ra_err**2 + dec_err**2 -
                              sqrt((ra_err**2 - dec_err**2)**2 +
                                   (2*ra_err*dec_err*ra_dec_corr)**2)))

M, m

(0.5212060177851401, 0.17049145733600912)

The program from Sebastien:

In [20]:
def error_ellipse(dx3, dy3, cor3):
    """
    """

    import numpy as np
    a = dx3
    b = dy3
    r = cor3
    rab = r*a*b
    theta = 0.5*np.arctan2(2*rab, (a**2-b**2))
    sx = a**2*np.cos(theta)**2+b**2*np.sin(theta)**2 + \
        2*rab*np.sin(theta)*np.cos(theta)
#     sy=a**2*np.cos(theta)**2+b**2*np.sin(theta)**2-2*rab*np.sin(theta)*np.cos(theta)
    sy = a**2*np.sin(theta)**2+b**2*np.cos(theta)**2 - \
        2*rab*np.sin(theta)*np.cos(theta)
    eema3 = np.sqrt(np.maximum(sx, sy))
    eena3 = np.sqrt(np.minimum(sx, sy))
    
    if theta < 0:
        theta0 = 360 + np.rad2deg(theta)
    else:
        theta0 = np.rad2deg(theta)  
        
    if a >= b:
        pa = 90 + np.rad2deg(theta)
    else:
        pa = (180 + np.rad2deg(theta)) % 180


    return eema3, eena3, pa


M, m, pa = error_ellipse(ra_err, dec_err, ra_dec_corr)

print("M = %.3f mas\n"
      "m = %.3f mas\n"
      "PA = %.0f degree" % (M, m, pa))

M = 0.521 mas
m = 0.170 mas
PA = 126 degree
